In [1]:
import pandas as pd
from main import test_one, test_two, test_three, test_four, test_five, test_six
import predict as pred
import warnings
warnings.filterwarnings('ignore')
# test_one()



error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\mod

In [20]:
def check_arguments(gold, predicated):

    for key in gold.keys():
        if key in predicated.keys():
            if gold[key].lower() == predicated[key].lower():
                continue
            else:
                return 0
        else:
            return 0
        
    return 1


# def check_long_distance(gold, pred):
#     patient = False
#     if "ARG0" in pred: # check if patient was predicted correctly
#         patient = pred["ARG0"]
    
#         gold_patient = gold["ARG0"]

#         if gold_patient.lower() in patient.lower():
#             patient = True
#         else:
#             return 0
#     else:
#         return 0
    
#     agent = False
#     if "ARG1" in pred: # check if agent was predicted correctly
#         agent = pred["ARG1"]
        
#         gold_agent = gold["ARG1"]

#         if gold_agent.lower() in agent.lower():
#             agent = True
#         else:
#             return 0
#     else:
#         return 0
    
#     if patient and agent:
#         return 1


def check_long_distance(gold_data, predicted_data):
    gold_patient = gold_data.get("ARG0", "").lower()
    predicted_patient = predicted_data.get("ARG0", "").lower()
    if gold_patient not in predicted_patient:
        return 0

    gold_agent = gold_data.get("ARG1", "").lower()
    predicted_agent = predicted_data.get("ARG1", "").lower()
    if gold_agent not in predicted_agent:
        return 0
    
    return 1



def test_seven():
    """
    Testing the SRL model on the test data for contextual infromation
    """
    df = pd.read_json("newtestdata/4_agent_patient_long_distance.json")
    tests = df['tests']
    print(f"Number of tests: {len(tests)}")

    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests:
        sentence = test['original_sentence']

        golden_labels = test['propbank_golden_labels']

        predicate = golden_labels['V']
        constituent, output_srl_bert = pred.predict_SRL_BERT(sentence, predicate)

        output_srl = pred.predict_SRL(sentence, predicate)
        output_group_9 = pred.predict_SRL_group9(sentence, constituent)
        # print(f"Original sentence: {sentence}")
        # print(f"Golden labels: {golden_labels}")
        # print(f"Predicted labels: {output_srl}")
        # print(f"Predicted labels BERT: {output_srl_bert}")
        # print(f"Predicted labels Group 9: {output_group_9}")

        del golden_labels['V']

        total_srl += check_long_distance(golden_labels, output_srl)
        total_srl_bert += check_long_distance(golden_labels, output_srl_bert)
        total_group_9 += check_long_distance(golden_labels, output_group_9)


    return pred.calculate_failure(len(tests), total_srl), pred.calculate_failure(len(tests), total_srl_bert), pred.calculate_failure(len(tests), total_group_9)

test_seven()

Number of tests: 10
Predicted labels: {'ARG0': 'The project , developed by the team of engineers who won the national competition ,', 'V': 'impressed', 'ARG1': 'the judges'}
Predicted labels: {'ARG1': 'The book , written by the author who was inspired by her childhood experiences ,', 'V': 'became', 'ARG2': 'a bestseller'}
Predicted labels: {'ARG1': 'The flowers , nurtured by the gardener with great care', 'V': 'bloomed', 'ARGM-MNR': 'beautifully'}
Predicted labels: {'ARG1': 'The cake , baked by the chef who had years of experience', 'V': 'was', 'ARG2': 'the highlight of the party'}
Predicted labels: {'ARG0': 'The computer program , designed by a team of experts who had worked on similar projects', 'V': 'solved', 'ARG1': 'the problem', 'ARGM-MNR': 'efficiently'}
Predicted labels: {'ARG0': 'The concert , organized by the committee who had been planning for months', 'V': 'attracted', 'ARG1': 'a large audience'}
Predicted labels: {'ARG1': 'The painting , created by the artist who had studi

(0.5, 0.5, 1.0)

In [19]:
l = "Hello i am jonas"

d = "jonass"

d.lower() in l.lower()

False